In [1]:
pip install elasticsearch

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
df = pd.read_csv("HorrorMoviesIMDb.csv")
df.head()

,Movie Title,Movie Year,Runtime,Genre,Rating,Director,Votes,Gross
0,Alien,1979,117,"Horror, Sci-Fi",8.5,Ridley Scott,"9,05,275",$78.90M
1,Psycho,1960,109,"Horror, Mystery, Thriller",8.5,Alfred Hitchcock,"6,89,068",$32.00M
2,The Shining,1980,146,"Drama, Horror",8.4,Stanley Kubrick,"10,51,582",$44.02M
3,The Thing,1982,109,"Horror, Mystery, Sci-Fi",8.2,John Carpenter,"4,39,793",$13.78M
4,Tumbbad,2018,104,"Drama, Fantasy, Horror",8.2,Rahi Anil Barve,"53,297",NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 836 entries, 0 to 835
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Movie Title  836 non-null    object 
 1   Movie Year   836 non-null    int64  
 2   Runtime      836 non-null    int64  
 3   Genre        836 non-null    object 
 4   Rating       836 non-null    float64
 5   Director     836 non-null    object 
 6   Votes        836 non-null    object 
 7   Gross        638 non-null    object 
dtypes: float64(1), int64(2), object(5)
memory usage: 52.4+ KB


In [4]:
from elasticsearch import Elasticsearch

import warnings
warnings.filterwarnings("ignore")

In [5]:
HOST = "http://localhost:9200"
INDEX_NAME = "desubatindex"

In [6]:
def connect_elasticsearch():
    es = None
    es = Elasticsearch([HOST])
    if es.ping():
        print("Bağlantı sağlandı")
    else:
        print("Bağlantı zaten varmış")
    
    return es
connect_elasticsearch()

Bağlantı sağlandı


<Elasticsearch(['http://localhost:9200'])>

In [10]:
"""
 0   Movie Title  836 non-null    object 
 1   Movie Year   836 non-null    int64  
 2   Runtime      836 non-null    int64  
 3   Genre        836 non-null    object 
 4   Rating       836 non-null    float64
 5   Director     836 non-null    object 
 6   Votes        836 non-null    object 
 7   Gross        638 non-null    object 
"""
mapping = {
    "Movie Title":
    {
        "type":"text"
    },
    "Movie Year":
    {
        "type":"integer"
    },
    "Runtime":
    {
        "type":"integer"
    },
    "Genre":
    {
        "type":"text"
    },
    "Rating":
    {
        "type":"float"
    },
    "Director":
    {
        "type":"text"
    },
    "Votes":
    {
        "type":"keyword"
    },
    "Gross":
    {
        "type":"keyword"
    }
}

def create_index():

    es = connect_elasticsearch()
    if es is not None:
        if not es.indices.exists(index=INDEX_NAME):
            res = es.indices.create(index=INDEX_NAME,body = {"mappings":{"properties":mapping}})

            print(res)
        else:
            print("İndex zaten varmış")
create_index()





Bağlantı sağlandı
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'desubatindex'}


In [11]:
def df_to_elasticsearch():
    es = connect_elasticsearch()
    df = pd.read_csv("HorrorMoviesIMDb.csv")
    df.dropna(inplace=True)
    df.reset_index(inplace=True)
    df_dict = df.to_dict("records")
    for i,row in enumerate(df_dict):
        res = es.index(index=INDEX_NAME,id=i,body = row)
        print(res["result"])
        
df_to_elasticsearch()


Bağlantı sağlandı
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
create

In [12]:
def get_data(sira_no):
    es = connect_elasticsearch()
    res = es.get(index=INDEX_NAME,id=sira_no)
    print(res)
get_data(sira_no=3)

Bağlantı sağlandı
{'_index': 'desubatindex', '_type': '_doc', '_id': '3', '_version': 1, '_seq_no': 3, '_primary_term': 1, 'found': True, '_source': {'index': 3, 'Movie Title': 'The Thing', 'Movie Year': 1982, 'Runtime': 109, 'Genre': 'Horror, Mystery, Sci-Fi', 'Rating': 8.2, 'Director': 'John Carpenter', 'Votes': '4,39,793', 'Gross': '$13.78M'}}


In [13]:
def get_data(sira_no):
    es = connect_elasticsearch()
    res = es.get(index=INDEX_NAME,id=sira_no)
    print(res["_source"])
get_data(sira_no=3)

Bağlantı sağlandı
{'index': 3, 'Movie Title': 'The Thing', 'Movie Year': 1982, 'Runtime': 109, 'Genre': 'Horror, Mystery, Sci-Fi', 'Rating': 8.2, 'Director': 'John Carpenter', 'Votes': '4,39,793', 'Gross': '$13.78M'}


* Match Query -> Belirli bir terimi veya belirli bir alanı içeren belgeleri aramak için kullanılır.

* Multi Match Query -> Belirli bir terimi birden çok terim içinde içeren belgeleri aramak için kullanılır

* Range Query -> Belirtilen belirli bir aralıkta yer alan belgeleri aramak için kullanılır.

* Aggregations -> Elasticsearch'teki verileri özetlemek ve gruplandırmak için kullanılır.

In [45]:
#Match Query
def match_query():
    es = connect_elasticsearch()

    query = {
            "query":{
                "match":{
                    'Movie Title': 'The'
                }
            }

    }

    res = es.search(index=INDEX_NAME,body = query)
    for i in res['hits']['hits']:
        print(i["_source"])
match_query()

Bağlantı sağlandı
{'index': 2, 'Movie Title': 'The Shining', 'Movie Year': 1980, 'Runtime': 146, 'Genre': 'Drama, Horror', 'Rating': 8.4, 'Director': 'Stanley Kubrick', 'Votes': '10,51,582', 'Gross': '$44.02M'}
{'index': 3, 'Movie Title': 'The Thing', 'Movie Year': 1982, 'Runtime': 109, 'Genre': 'Horror, Mystery, Sci-Fi', 'Rating': 8.2, 'Director': 'John Carpenter', 'Votes': '4,39,793', 'Gross': '$13.78M'}
{'index': 5, 'Movie Title': 'The Exorcist', 'Movie Year': 1973, 'Runtime': 122, 'Genre': 'Horror', 'Rating': 8.1, 'Director': 'William Friedkin', 'Votes': '4,22,330', 'Gross': '$232.91M'}
{'index': 22, 'Movie Title': 'The Innocents', 'Movie Year': 1961, 'Runtime': 100, 'Genre': 'Horror', 'Rating': 7.8, 'Director': 'Jack Clayton', 'Votes': '31,689', 'Gross': '$2.62M'}
{'index': 30, 'Movie Title': 'The Fly', 'Movie Year': 1986, 'Runtime': 96, 'Genre': 'Drama, Horror, Sci-Fi', 'Rating': 7.6, 'Director': 'David Cronenberg', 'Votes': '1,91,571', 'Gross': '$40.46M'}
{'index': 32, 'Movie Ti

In [21]:
#Multi Match Query 

#Match Query
def multi_match_query():
    es = connect_elasticsearch()

    query = {
            "query":{
                "multi_match":{
                    'query': 'Horror',
                    "fields":["Movie Title","Director"]
                }
            }

    }

    res = es.search(index=INDEX_NAME,body = query)
    for i in res['hits']['hits']:
        print(i["_source"])
multi_match_query()

Bağlantı sağlandı
{'index': 483, 'Movie Title': 'The Amityville Horror', 'Movie Year': 1979, 'Runtime': 117, 'Genre': 'Horror', 'Rating': 6.1, 'Director': 'Stuart Rosenberg', 'Votes': '41,808', 'Gross': '$86.43M'}
{'index': 529, 'Movie Title': 'The Amityville Horror', 'Movie Year': 2005, 'Runtime': 90, 'Genre': 'Horror', 'Rating': 5.9, 'Director': 'Andrew Douglas', 'Votes': '1,14,596', 'Gross': '$65.23M'}
{'index': 60, 'Movie Title': 'The Rocky Horror Picture Show', 'Movie Year': 1975, 'Runtime': 100, 'Genre': 'Comedy, Horror, Musical', 'Rating': 7.4, 'Director': 'Jim Sharman', 'Votes': '1,58,254', 'Gross': '$139.88M'}


In [26]:
# Range Query
def range_query():
    es = connect_elasticsearch()

    query = {
            "query":{
                "bool":{
                    "must":[
                        {
                            "range":{
                                'Movie Year':{"gt":1998,"lt":2005}
                            }
                         }
                    ]
                }
            }

    }

    res = es.search(index=INDEX_NAME,body = query)
    for i in res['hits']['hits']:
        print(i["_source"])

range_query()

Bağlantı sağlandı
{'index': 11, 'Movie Title': 'Shaun of the Dead', 'Movie Year': 2004, 'Runtime': 99, 'Genre': 'Comedy, Horror', 'Rating': 7.9, 'Director': 'Edgar Wright', 'Votes': '5,72,237', 'Gross': '$13.54M'}
{'index': 27, 'Movie Title': 'American Psycho', 'Movie Year': 2000, 'Runtime': 102, 'Genre': 'Crime, Drama, Horror', 'Rating': 7.6, 'Director': 'Mary Harron', 'Votes': '6,58,696', 'Gross': '$15.07M'}
{'index': 28, 'Movie Title': 'Saw', 'Movie Year': 2004, 'Runtime': 103, 'Genre': 'Horror, Mystery, Thriller', 'Rating': 7.6, 'Director': 'James Wan', 'Votes': '4,36,144', 'Gross': '$56.00M'}
{'index': 34, 'Movie Title': 'The Others', 'Movie Year': 2001, 'Runtime': 104, 'Genre': 'Horror, Mystery, Thriller', 'Rating': 7.6, 'Director': 'Alejandro Amenábar', 'Votes': '3,77,287', 'Gross': '$96.52M'}
{'index': 37, 'Movie Title': 'Vampire Hunter D: Bloodlust', 'Movie Year': 2000, 'Runtime': 103, 'Genre': 'Animation, Action, Drama', 'Rating': 7.6, 'Director': 'Jack Fletcher', 'Votes': '3

In [27]:
df.describe()

,Movie Year,Runtime,Rating
count,836.000000,836.000000,836.000000
mean,2004.271531,100.767943,6.160167
std,16.330837,14.484556,0.940355
min,1920.000000,57.000000,1.600000
25%,1998.000000,91.000000,5.600000
50%,2009.000000,98.000000,6.200000
75%,2016.000000,108.000000,6.800000
max,2023.000000,191.000000,8.500000


In [34]:
# agg Query
def agg_query():
    es = connect_elasticsearch()

    query = {
            "aggs":{
                "movie_desc":{
                    "stats":{
                            "field":"Rating"
                    }
                }
            }

    }

    res = es.search(index=INDEX_NAME,body = query)
    # for i in res['aggregations']:
    #     print(i["_source"])

    print(res)

agg_query()

Bağlantı sağlandı
{'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 638, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'desubatindex', '_type': '_doc', '_id': '0', '_score': 1.0, '_source': {'index': 0, 'Movie Title': 'Alien', 'Movie Year': 1979, 'Runtime': 117, 'Genre': 'Horror, Sci-Fi', 'Rating': 8.5, 'Director': 'Ridley Scott', 'Votes': '9,05,275', 'Gross': '$78.90M'}}, {'_index': 'desubatindex', '_type': '_doc', '_id': '1', '_score': 1.0, '_source': {'index': 1, 'Movie Title': 'Psycho', 'Movie Year': 1960, 'Runtime': 109, 'Genre': 'Horror, Mystery, Thriller', 'Rating': 8.5, 'Director': 'Alfred Hitchcock', 'Votes': '6,89,068', 'Gross': '$32.00M'}}, {'_index': 'desubatindex', '_type': '_doc', '_id': '2', '_score': 1.0, '_source': {'index': 2, 'Movie Title': 'The Shining', 'Movie Year': 1980, 'Runtime': 146, 'Genre': 'Drama, Horror', 'Rating': 8.4, 'Director': 'Stanley Kubrick', 'Votes': '10,51,5

In [46]:
#Match Query
def match_query():
    es = connect_elasticsearch()

    query = {
            "query":{
                "match":{
                    'Movie Title': 'The'
                }
            },
            "size":1000

    }

    res = es.search(index=INDEX_NAME,body = query)
    for i in res['hits']['hits']:
        print(i["_source"])
match_query()

Bağlantı sağlandı
{'index': 2, 'Movie Title': 'The Shining', 'Movie Year': 1980, 'Runtime': 146, 'Genre': 'Drama, Horror', 'Rating': 8.4, 'Director': 'Stanley Kubrick', 'Votes': '10,51,582', 'Gross': '$44.02M'}
{'index': 3, 'Movie Title': 'The Thing', 'Movie Year': 1982, 'Runtime': 109, 'Genre': 'Horror, Mystery, Sci-Fi', 'Rating': 8.2, 'Director': 'John Carpenter', 'Votes': '4,39,793', 'Gross': '$13.78M'}
{'index': 5, 'Movie Title': 'The Exorcist', 'Movie Year': 1973, 'Runtime': 122, 'Genre': 'Horror', 'Rating': 8.1, 'Director': 'William Friedkin', 'Votes': '4,22,330', 'Gross': '$232.91M'}
{'index': 22, 'Movie Title': 'The Innocents', 'Movie Year': 1961, 'Runtime': 100, 'Genre': 'Horror', 'Rating': 7.8, 'Director': 'Jack Clayton', 'Votes': '31,689', 'Gross': '$2.62M'}
{'index': 30, 'Movie Title': 'The Fly', 'Movie Year': 1986, 'Runtime': 96, 'Genre': 'Drama, Horror, Sci-Fi', 'Rating': 7.6, 'Director': 'David Cronenberg', 'Votes': '1,91,571', 'Gross': '$40.46M'}
{'index': 32, 'Movie Ti